# ETL 

In [78]:
#Import necessary libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
import re


In [79]:
base = Path("..")  
csv_files = list(base.rglob("*.csv"))

print("Found CSV files:")
for file in csv_files:
    print(f"  {file}")
    


Found CSV files:
  ..\data\raw\companies\companies.csv
  ..\data\raw\companies\company_industries.csv
  ..\data\raw\companies\company_specialities.csv
  ..\data\raw\companies\employee_counts.csv
  ..\data\raw\jobs\benefits.csv
  ..\data\raw\jobs\job_industries.csv
  ..\data\raw\jobs\job_skills.csv
  ..\data\raw\jobs\salaries.csv
  ..\data\raw\mappings\industries.csv
  ..\data\raw\mappings\skills.csv
  ..\data\transformed\companies\it_companies.csv
  ..\data\transformed\companies\it_company_industries.csv
  ..\data\transformed\companies\it_employee_counts.csv
  ..\data\transformed\companies\it_specialities.csv
  ..\data\transformed\jobs\it_benefits.csv
  ..\data\transformed\jobs\it_job_industries_cleaned.csv
  ..\data\transformed\jobs\it_job_skills.csv
  ..\data\transformed\jobs\it_salaries.csv
  ..\data\transformed\mappings\industries_it_only.csv


#### 1. Filter IT Companies from `company_industries.csv`
Extracting only companies in the "IT Services and IT Consulting" industry to get relevant `company_id`s for our analysis.


In [80]:
input_path = "../data/raw/companies/company_industries.csv"
output_path = "../data/transformed/companies/it_company_industries.csv"

# Ensure output directory exists
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Load the data
df = pd.read_csv(input_path)

# Check available columns
print(df.columns)

# Filter for IT Services and IT Consulting
it_df = df[df['industry'] == 'IT Services and IT Consulting']

# Save filtered data
it_df.to_csv(output_path, index=False)

print(f"Filtered data saved to: {output_path}")
print("Remaining rows:", it_df.shape[0])

Index(['company_id', 'industry'], dtype='object')
Filtered data saved to: ../data/transformed/companies/it_company_industries.csv
Remaining rows: 2130


#### 2. Filter related company data
Using `company_id`s from the filtered IT companies to extract relevant records from:
- `raw/companies/companies.csv`
- `raw/companies/company_specialities.csv`
- `raw/companies/employee_counts.csv`


In [81]:
# Load company_ids of IT companies
it_ids = pd.read_csv("../data/transformed/companies/it_company_industries.csv")["company_id"]

# Define input/output paths
input_files = {
    "companies": "../data/raw/companies/companies.csv",
    "specialities": "../data/raw/companies/company_specialities.csv",
    "employee_counts": "../data/raw/companies/employee_counts.csv"
}

output_base = "../data/transformed/companies"

# Filter and save each dataset
for name, path in input_files.items():
    df = pd.read_csv(path)
    
    # Filter rows based on company_id
    filtered_df = df[df["company_id"].isin(it_ids)]
    
    # Ensure output directory exists
    os.makedirs(output_base, exist_ok=True)
    
    # Save the filtered data
    out_path = f"{output_base}/it_{name}.csv"
    filtered_df.to_csv(out_path, index=False)
    
    print(f"{name} filtered → {filtered_df.shape[0]} rows saved to {out_path}")


companies filtered → 2130 rows saved to ../data/transformed/companies/it_companies.csv
specialities filtered → 19623 rows saved to ../data/transformed/companies/it_specialities.csv
employee_counts filtered → 3345 rows saved to ../data/transformed/companies/it_employee_counts.csv


#### 3. Transform `time_recorded` in `it_employee_counts.csv`
The `time_recorded` column contains Unix timestamps. We'll convert them into human-readable datetime format.


In [82]:
from datetime import datetime

# Load filtered employee counts data
emp_df = pd.read_csv("../data/transformed/companies/it_employee_counts.csv")

# Convert Unix timestamp to datetime
emp_df["time_recorded"] = pd.to_datetime(emp_df["time_recorded"], unit="s")

# Save the updated DataFrame
emp_df.to_csv("../data/transformed/companies/it_employee_counts.csv", index=False)

print(" 'time_recorded' converted to datetime.")
print(emp_df[["company_id", "time_recorded"]].head())


 'time_recorded' converted to datetime.
   company_id       time_recorded
0      729238 2024-04-05 19:42:53
1     2934678 2024-04-05 19:42:53
2    15984730 2024-04-05 19:42:53
3     6618000 2024-04-05 19:42:53
4    75056372 2024-04-05 19:42:53


#### 4. Transforming Industries Dataset: Filtering for IT-Related Roles

To identify job postings relevant to IT and tech, we'll apply a mapping filter to the `industries` field. This step is crucial because some industries lack proper names and are only identifiable by keywords. We'll use two sets of patterns:

- **Inclusion patterns** (indicating relevance to IT)
- **Exclusion patterns** (to avoid false positives from overlapping terms)

In [83]:
# Load raw industry mappings
industry_path = "../data/raw/mappings/industries.csv"
industries_df = pd.read_csv(industry_path)

# Define IT-related inclusion and exclusion patterns
it_patterns = [
    r"\bIT\b",
    r"\bInformation Technology\b",
    r"\bSoftware\b",
    r"\bDeveloper\b",
    r"\bEngineer\b",
    r"\bProgrammer\b",
    r"\bTech\b",
    r"\bData\b",
    r"\bCloud\b",
    r"\bSystem\b",
    r"\bNetwork\b"
]

exclude_patterns = [
    r"\bBuilding\b",
    r"\bHVAC\b",
    r"\bMaintenance\b",
    r"\bFacilities?\b",
    r"\bStaff\b",
    r"\bConstruction\b",
    r"\bMechanical\b",
    r"\bElectrical\b",
    r"\bCivil\b",
    r"\bManufacturing\b",
    r"\bProject Manager\b",
    r"\bProject Engineer\b",
    r"\bQuality\b",
    r"\bSafety\b",
    r"\bLogistics\b",
    r"\bSupply Chain\b",
    r"\bField\b",
    r"\bService\b",
    r"\bSupport\b", 
    r"\bStructural\b",
    r"\bProcess\b", 
    r"\bDesign\b",
    r"\bInfrastructure\b",
    r"\bEnergy\b",
]

# Function to determine if an industry name is IT-related
def is_it_industry(industry_name):
    if not isinstance(industry_name, str):
        return False
    name = industry_name.lower()
    include = any(re.search(pat, name, flags=re.IGNORECASE) for pat in it_patterns)
    exclude = any(re.search(pat, name, flags=re.IGNORECASE) for pat in exclude_patterns)
    return include and not exclude

# Apply the filter
industries_df["is_it"] = industries_df["industry_name"].apply(is_it_industry)
it_industries = industries_df[industries_df["is_it"]].drop(columns=["is_it"])

# Save transformed output
output_path = "../data/transformed/mappings/industries_it_only.csv"
it_industries.to_csv(output_path, index=False)

print(f"Filtered IT-related industries saved to: {output_path}")

Filtered IT-related industries saved to: ../data/transformed/mappings/industries_it_only.csv


#### 5. Filter job IDs by IT-related industries

Using the filtered `industry_id`s from the transformed industries dataset, we filter job IDs from `../data/raw/jobs/job_industries.csv` to retain only jobs belonging to IT-related industries.

We then use these filtered `job_id`s to subset the following datasets:

- `../data/raw/jobs/benefits.csv`
- `../data/raw/jobs/job_skills.csv`
- `../data/raw/jobs/salaries.csv`

Only rows with `job_id`s present in the IT subset are retained.


In [84]:
# Paths
job_industries_path = Path("../data/raw/jobs/job_industries.csv")
benefits_path = Path("../data/raw/jobs/benefits.csv")
skills_path = Path("../data/raw/jobs/job_skills.csv")
salaries_path = Path("../data/raw/jobs/salaries.csv")
output_dir = Path("../data/transformed/jobs")
output_dir.mkdir(parents=True, exist_ok=True)

# Load job_industries.csv
job_industries_df = pd.read_csv(job_industries_path)

# Load the filtered IT-related industries
it_industries_path = Path("../data/transformed/mappings/industries_it_only.csv")
it_industries_df = pd.read_csv(it_industries_path)

# Filter job IDs based on IT-related industry IDs
it_industry_ids = set(it_industries_df["industry_id"])
filtered_job_ids = job_industries_df[job_industries_df["industry_id"].isin(it_industry_ids)]["job_id"].unique()

# Save filtered job_industries.csv
job_industries_df[job_industries_df["job_id"].isin(filtered_job_ids)]\
    .to_csv(output_dir / "it_job_industries.csv", index=False)

# Filter and save other job datasets
pd.read_csv(benefits_path).query("job_id in @filtered_job_ids")\
    .to_csv(output_dir / "it_benefits.csv", index=False)

pd.read_csv(skills_path).query("job_id in @filtered_job_ids")\
    .to_csv(output_dir / "it_job_skills.csv", index=False)

pd.read_csv(salaries_path).query("job_id in @filtered_job_ids")\
    .to_csv(output_dir / "it_salaries.csv", index=False)

#### 6. Check if industry_id and job_id are linked correctly

In [85]:
# Load datasets
industries_df = pd.read_csv("../data/transformed/mappings/industries_it_only.csv")
job_industries_df = pd.read_csv("../data/transformed/jobs/it_job_industries.csv")

# Rename 'name' to 'industry_name' if needed
if 'name' in industries_df.columns and 'industry_name' not in industries_df.columns:
    industries_df = industries_df.rename(columns={"name": "industry_name"})

# Merge to inspect job_id with industry info
merged_df = job_industries_df.merge(industries_df, on="industry_id", how="left")

# Keep only job_ids with valid industry_id matches
clean_df = merged_df[merged_df["industry_name"].notna()]

# Save the cleaned data
clean_df.to_csv("../data/transformed/jobs/it_job_industries_cleaned.csv", index=False)
print(clean_df[["job_id", "industry_id", "industry_name"]].head())

# Delete the old unclean file
old_path = "../data/transformed/jobs/it_job_industries.csv"
if os.path.exists(old_path):
    os.remove(old_path)
    print(f"Deleted old file: {old_path}")
else:
    print(f"File not found, nothing to delete: {old_path}")

       job_id  industry_id                  industry_name
0  3884431567           96  IT Services and IT Consulting
1  3884916106           96  IT Services and IT Consulting
2  3884916106            4           Software Development
4  3884431568           96  IT Services and IT Consulting
5  3861704803           96  IT Services and IT Consulting
Deleted old file: ../data/transformed/jobs/it_job_industries.csv
